<a href="https://colab.research.google.com/github/raviakasapu/LLM-Training-Docs/blob/main/LLM_DeciCoder_1b_Code_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM DeciCoder-1b code generation

https://huggingface.co/Deci/DeciCoder-1b

https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard


Reasons for selecting DeciCoder-1b
* small LLM for testing and prototyping
* Active developer usage
* Better benchmark score for GSM8K of 1.74 (ability to solve multi-step mathematical reasoning problems) (Considered pythia-1.3b otherwise due to better score)
*

In [1]:
!pip -q install langchain huggingface_hub transformers sentence_transformers xformers

In [2]:
!pip install bitsandbytes>=0.39.0 -q

In [3]:
# pip install -q transformers
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from transformers import pipeline

from typing import Union, Tuple, List
import os
import torch


In [4]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = <hf token here>
token = os.environ['HUGGINGFACEHUB_API_TOKEN']

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.ca

In [6]:
checkpoint = "Deci/DeciCoder-1b"
device = "cuda" # for GPU usage or "cpu" for CPU usage

In [7]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_fim = AutoTokenizer.from_pretrained(checkpoint, padding_side="left")

FIM_PREFIX = "<fim_prefix>"
FIM_MIDDLE = "<fim_middle>"
FIM_SUFFIX = "<fim_suffix>"
FIM_PAD = "<fim_pad>"
EOD = "<|endoftext|>"


tokenizer_fim.add_special_tokens({
  "additional_special_tokens": [EOD, FIM_PREFIX, FIM_MIDDLE, FIM_SUFFIX, FIM_PAD],
  "pad_token": EOD,
})

0

In [8]:
prompt = "def all_odd_elements(sequence):\n    \"\"\"Returns every odd element of the sequence.\"\"\""

In [9]:
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer([prompt], return_tensors="pt",padding=True).to(device)

In [10]:
tokenizer.padding_side

'left'

In [11]:
model = AutoModelForCausalLM.from_pretrained("Deci/DeciCoder-1b", trust_remote_code=True)

input_text = "<fim-prefix>def fib(n):<fim-suffix>    else:\n        return fib(n - 2) + fib(n - 1)<fim-middle>"

inputs = tokenizer_fim(input_text, return_tensors="pt", padding=True, return_token_type_ids=False)

outputs = model.generate(**inputs, max_new_tokens=25)

generation = [tokenizer_fim.decode(tensor, skip_special_tokens=False) for tensor in outputs]

print(generation[0])


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim-prefix>def fib(n):<fim-suffix>    else:
        return fib(n - 2) + fib(n - 1)<fim-middle>
    <fim-prefix>def fib(n):<fim-suffix>    else:
        return fib(


In [68]:
def read_until_empty_line(inputString):
  result = []
  final_result = ''
  lines = inputString.splitlines()
  for line in lines:
    if(line==""):
      break
    final_result = final_result+line+"\n"
  return final_result


In [70]:
input_text = "<fim_prefix>def print_hello_world():\n    <fim_suffix>\n    print('Hello world!')<fim_middle>"
inputs = tokenizer_fim.encode(input_text, return_tensors="pt")
outputs = model.generate(inputs)
result = tokenizer.decode(outputs[0])

print(read_until_empty_line(result))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<fim_prefix>def print_hello_world():
    <fim_suffix>
    print('Hello world!')<fim_middle><|endoftext|>



In [71]:
prompt = "def all_odd_elements(sequence):\n    \"\"\"Returns every odd element of the sequence.\"\"\""
inputs = tokenizer_fim.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs,max_new_tokens=50)
result = tokenizer.decode(outputs[0])

print(read_until_empty_line(result))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


def all_odd_elements(sequence):
    """Returns every odd element of the sequence."""
    return [x for x in sequence if x % 2!= 0]



In [72]:
prompt = "def end_of_line(sequence):\n    \"\"\"Returns text till empty line.\"\"\""
inputs = tokenizer_fim.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs,max_new_tokens=50)
result = tokenizer.decode(outputs[0])

print(read_until_empty_line(result))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


def end_of_line(sequence):
    """Returns text till empty line."""
    return sequence.rsplit('\n', 1)[0]

